In [ ]:
##############################################################################
###################### Making necessary library imports ###################### 
##############################################################################

import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import matplotlib.image as mpimg
from scipy import optimize
from skimage import feature
import random
import collections

In [ ]:
###############################################################################
########################## Task 1  ############################################
###############################################################################

# Function to get an initial estimate of Fundamental Matrix
def estimate_fundamental_matrix(points1,points2,n):

    # declare empty A matrix of size n X 9
    # n is the number of point correspondences
    A_matrix =  np.zeros((n,9))
    for i in range(n):
        P_1 = points1[i]
        P_2 = points2[i]

        # x and y coordinates of point in the left image
        x = P_1[0]
        y = P_2[1]

        # x and y coordinates of point in the right image
        x_dash = P_1[0]
        y_dash = P_2[1]

        # construct the A matrix
        A_matrix[i,:] = np.array(([x_dash*x,x_dash*y,x_dash,y_dash*x,y_dash*y,y_dash,x,y,1]))

    # compute svd
    U,S,V = np.linalg.svd(A_matrix)
    h = V[8]
    F = h.reshape(3,3)

    # return the fundamental matrix
    return F

# Function to enforce rank constraint on Fundamental Matrix
def condition_F(input_F):

    U,S,V = np.linalg.svd(input_F)

    # this step conditions the fundamental matrix
    S[2] = 0
    D = np.dot(np.diag(S),V)
    F = np.dot(U,D)

    # return the rank 2 fundamental matrix
    return F

# Function to estimate both epipoles from Fundamental matrix
def estimate_epipoles(F):

    U,S,V = np.linalg.svd(F)

    # left epipole
    e = np.transpose(V[2,:])

    # right epipole
    e_dash = U[:,2]

    # return both epipoles
    return e,e_dash

# Function to get left and right projection matrices in canonical configuration
def estimate_projection_canonical(e_dash,F):

    # left camera projection matrix
    P = np.array(([1,0,0,0],[0,1,0,0],[0,0,1,0]))

    # right camera projection matrix
    P_dash = np.zeros((3,4))
    s = np.array(([0,(-1)*e_dash[2],e_dash[1]],
                 [e_dash[2],0,(-1)*e_dash[0]],
                 [(-1)*e_dash[1],e_dash[0],0]))
    M = np.dot(s,F)
    P_dash[:,3] = e_dash
    P_dash[0:3,0:3] = M

    # return projection matrices in canonical form
    return P,P_dash
    
# Function to mark points on an image given coordinates
# Taken from HW2 and HW3 submissions
def draw_bounding_box(image,x,y):
    
    # Display the image
    # x is the list of x coordinates of the points that need to be marked
    # y is the list of y coordinates of the points that need to be marked
    plt.figure(figsize=(6,6))
    fig = plt.imshow(image)
    for i in range(len(x)):
        plt.plot(x[i],y[i],marker='.',color="red")
        plt.text(x[i],y[i],str(i+1),horizontalalignment='right',color='white',fontsize=12)
    plt.axis('off')

    # to show the plot
    plt.show()

# Function for calculating cost function for non-linear optimization
# Inspired/Derived from previous 2020 HW submissions
def compute_cost_function(f,x1,x2,y1,y2,P,P_dash):

    # Reshape fundamental matrix vector to a 3 X 3 matrix
    F = f.reshape(3,3)

    # compute left and right epipoles using F matrix
    e,e_dash = estimate_epipoles(F)

    # normalize left epipole
    e = e/e[2]

    # normalize right epipole
    e_dash = e_dash/e_dash[2]

    # obtain left and right projection matrices in canonical form
    P,P_dash = estimate_projection_canonical(e_dash,F)

    e = []
    for i in range(len(x1)):

        # transform 2-D points to 3-D points using projection matrices
        D = np.zeros((4,4))
        D[0] = x1[i]*P[2] 
        D[0] = D[0] - P[0]
        D[1] = y1[i]*P[2] 
        D[1] = D[1] - P[1]
        D[2] = x2[i]*P_dash[2]
        D[2] = D[2] - P_dash[0]
        D[3] = y2[i]*P_dash[2] 
        D[3] = D[3] - P_dash[1]
        U,S,V = np.linalg.svd(D)
        X = np.transpose(V[3])
        X = X/X[3]

        # project world coordinate using left camera
        projected_coord_1 = np.dot(P,X)

        # normalize in homogeneous coordinates
        projected_coord_1 = projected_coord_1/projected_coord_1[2]

        # project world coordinate using right camera
        projected_coord_2 = np.dot(P_dash,X)

        # normalize in homogeneous coordinates
        projected_coord_2 = projected_coord_2/projected_coord_2[2]

        # computer geometric error for left camera
        # between actual pixel and projected pixel
        points1_hom = np.array(([projected_coord_1[0],projected_coord_1[1],1]))
        points1 = np.array(([x1[i],y1[i],1]))
        error_img_1 = np.linalg.norm(points1 - points1_hom)
        error_img_1 = np.square(error_img_1)
        e.append(error_img_1)

        # computer geometric error for right camera
        # between actual pixel and projected pixel
        points2_hom = np.array(([projected_coord_2[0],projected_coord_2[1],1]))
        points2 = np.array(([x2[i],y2[i],1]))
        error_img_2 = np.linalg.norm(points2 - points2_hom)
        error_img_2 = np.square(error_img_2)
        e.append(error_img_2)
    overall_error = np.ravel(e)

    # return total error for non-linear optimization
    return overall_error

# Function for estimating left and right homographies for rectification
# Inspired/Derived from previous 2020 HW submissions
def computing_homographies(H,W,P,P_dash,F,e,e_dash,x1,x2,y1,y2):

    # H: height of any one of the images
    # W: width of any one of the images
    # P,P_dash: left and right camera projection matrices 
    # F: refined and conditioned fundamental matrix
    # e,e_dash: left and right epipoles
    # x1,y1: point correspondences in left image
    # x2,y2: point correspondences in right image

    # Right Image
    # For computing H_dash, we need to compute G,R,T matrices first
    # compute angle and other needed parameters
    theta1 = e_dash[1] - (H/2)
    theta2 = e_dash[0] - (W/2)
    theta = math.atan2(theta1,-theta2)
    f1 = np.cos(theta)*(e_dash[0] - (W/2))
    f2 = np.sin(theta)*(e_dash[1] - (H/2))
    f = f1 - f2
    c = np.cos(theta) 
    s = np.sin(theta)

    # computing G
    G = np.array(([1,0,0],[0,1,0],[-1/f,0,1]))
    # computing R
    R = np.array(([c,-s,0],[s,c,0],[0,0,1]))
    # compute T
    T = np.array(([1,0,-W/2],[0,1,-H/2],[0,0,1]))

    # compute right homography H_dash
    H_dash = np.dot(R,T)
    H_dash = np.dot(G,H_dash)
    # normalize right homography
    H_dash = H_dash/H_dash[2,2]

    # Apply right homography to center of right image
    center_2 = np.array([W/2,H/2,1]) 
    new_center_2 = np.dot(H_dash,center_2)
    # Normalize
    new_center_2 = new_center_2/new_center_2[2]

    # compute right translation matrix w.r.t center
    trans_2 = np.array(([1,0,(W/2) - new_center_2[0]],[0,1,(H/2) - new_center_2[1]],[0,0,1]))
    H_dash_new = np.dot(trans_2,H_dash)
    # H_dash_new is the normalized final right homography
    H_dash_new = H_dash_new /H_dash_new [2,2]

    # Left Image
    # For computing H, we need to compute G,R,T matrices first
    # compute angle and other needed parameters
    theta1 = e[1] - (H/2)
    theta2 = e[0] - (W/2)
    theta = math.atan2(theta1,-theta2)
    f1 = np.cos(theta)*(e[0] - (W/2))
    f2 = np.sin(theta)*(e[1] - (H/2))
    f = f1 - f2
    c = np.cos(theta) 
    s = np.sin(theta)

    # computing G
    G = np.array(([1,0,0],[0,1,0],[-1/f,0,1]))
    # computing R
    R = np.array(([c,-s,0],[s,c,0],[0,0,1]))
    # compute T
    T = np.array(([1,0,-W/2],[0,1,-H/2],[0,0,1]))

    # compute initial left homography H
    H_left_init = np.dot(R,T)
    H_left_init = np.dot(G,H_left_init)

    # compute cost to minimize distance
    A = np.zeros((len(x1),3))
    B = np.zeros((len(x1),3))
    for i in range(len(x1)):
        point1 = np.array(([x1[i],y1[i],1]))
        point2 = np.array(([x2[i],y2[i],1]))
        trans_point_1 = np.dot(H_left_init,point1)
        trans_point_2 = np.dot(H_dash_new,point2)
        trans_point_1 = trans_point_1/trans_point_1[2]
        trans_point_2 = trans_point_2/trans_point_2[2]
        A[i,:] = trans_point_1
        B[i,:] = trans_point_2

    # using linear least squares
    A_inv = np.linalg.pinv(A)
    C = np.dot(A_inv,B[:,0])
    a = C[0]; b = C[1]; c = C[2]
    H_a = np.array(([a,b,c],[0,1,0],[0,0,1]))

    # Estimate of left homography that will rectify the center of left image
    H_left_c = np.dot(H_a,H_left_init)

    # Apply left homography to center of left image
    # compute center of left image
    center_1 = np.array([W/2,H/2,1]) 
    new_center_1 = np.dot(H_left_c,center_1)

    # normalize the rectified left image center
    new_center_1 = new_center_1/new_center_1[2]

    # compute left translation matrix w.r.t center
    trans_1 = np.array(([1,0,(W/2) - new_center_1[0]],[0,1,(H/2) - new_center_1[1]],[0,0,1]))
    H_new = np.dot(trans_1,H_left_c)
    # H_new is the normalized final left homography
    H_new = H_new / H_new[2,2]

    # return both left and right homographies
    return H_new,H_dash_new

In [ ]:
# Function to convert a RGB image to grayscale
# Taken from HW8 submission
def convert_rgb_grayscale(input_rgb):
    
    grayscale_image = cv2.cvtColor(input_rgb, cv2.COLOR_BGR2GRAY)

    return grayscale_image

# Function to detect edges using Canny Edge Detector
# Taken from HW8 submission
def canny_edge_detector(input_image):
    
    # convert rgb to grayscale image
    input_image = convert_rgb_grayscale(input_image)

    # extract edges using Canny Edge Detector of Skimage
    edges_image = feature.canny(input_image,sigma=3)

    # plot the image with detected edges
    plt.figure()
    plt.imshow(edges_image,cmap="gray")
    plt.axis('off')

    # return the image with edges
    return edges_image

# Function to find point correspondences after rectification
# Inspired/Derived from previous 2020 HW submissions
def find_correspondences(edges_image_1,edges_image_2):

    correspondence_list_1 = []
    correspondence_list_2 = []

    #compute the number of rows
    num_rows = edges_image_1.shape[0]
    for i in range(num_rows):

        # check where pixels are non-zero i.e. pixels detected by canny edge detector
        non_zero_pos = np.where(edges_image_1[i] > 0 )

        # detect the row coordinate of that non-zero pixel
        for j in non_zero_pos[0]:
            start_column = j
            end_column = j + 30

            # search within same row for the right image in a range of columns
            cols_region = edges_image_2[i,start_column:end_column ]
            if np.size(np.where(cols_region>0)[0]) != 0:
                k = j+ np.where(cols_region>0)[0][0]
                edges_image_2[i,k] = 0

                # save point correspondences of both rectified images 
                correspondence_list_1.append([j,i])
                correspondence_list_2.append([k,i])
    return correspondence_list_1,correspondence_list_2

# Functions to use SSD metric to find correspondence matches 
# Borrowed from HW4 submission 
def compute_ssd(img1,img2,corners_for_image_1,corners_for_image_2):

    '''
    Input  - img1                : Grayscale first image
           - img2                : Grayscale second image
           - corners_for_image_1 : Detected corners using Harris for Image 1
           - corners_for_image_2 : Detected corners using Harris for Image 2
    Output - coordinate_array_1_top: Selected corners in first image after SSD 
           - coordinate_array_2_top: Selected corners in second image after SSD 
    '''

    #Size of window chosen m = 6
    m = 6

    value_list = []
    coordinate_list_1 = []
    coordinate_list_2 = []

    # Normalize grayscale images 1 and 2
    img_1_norm = img1/np.max(img1)
    img_2_norm = img2/np.max(img2)

    # extract a (m+1) X (m+1) patch around the corners in both images and save them to two arrays
    # apply SSD formula to both arrays
    # Repeat for all corners and choose top 200 corner pairs which give least SSD
    for i in range(len(corners_for_image_1)):
        for j in range(len(corners_for_image_2)):

            point_1 = corners_for_image_1[i]
            point_2 = corners_for_image_2[j]

            array_1 = np.zeros((m+1,m+1))
            array_2 = np.zeros((m+1,m+1))

            if point_1[0] - m > 0 :
                if point_1[0] < img1.shape[0] - m - 1:
                    array_1 = img_1_norm[point_1[0] - int(m/2):point_1[0]+int((m+2)/2), point_1[1] - int(m/2):point_1[1]+int((m+2)/2)]
            if point_2[0] - m > 0 :
                if point_2[0] < img2.shape[0] - m - 1:
                    array_2 = img_2_norm[point_2[0] - int(m/2):point_2[0]+int((m+2)/2), point_2[1] - int(m/2):point_2[1]+int((m+2)/2)]
            
            # applying SSD criterion
            if array_1.shape == array_2.shape:
                value = np.sum(np.square(np.abs(array_1-array_2)))
                value_list.append(value)
                coordinate_list_1.append(point_1)
                coordinate_list_2.append(point_2)

    value_array = np.array(value_list)

    # get all unique SSD distances
    value_array,value_array_index = np.unique(value_array,return_index=True,return_inverse=False,return_counts=False, axis=None)
    coordinate_array_1 = np.array(coordinate_list_1)[value_array_index]
    coordinate_array_2 = np.array(coordinate_list_2)[value_array_index]

    # sort the distances in ascending order
    value_sorted_indices = np.argsort(value_array)

    # choose top 200 minimum distances and select those corners
    value_top_100 =  value_sorted_indices[0:200]
    coordinate_array_1_top =  coordinate_array_1[value_top_100]
    coordinate_array_2_top =  coordinate_array_2[value_top_100]
    return coordinate_array_1_top,coordinate_array_2_top
            
# Function to draw point correspondences given matches and two images
# Borrowed from HW4 submission 
def draw_matches_harris(corners_1,corners_2,img1,img2,name):

    # stack the image pair together
    overall_image = np.hstack((img1,img2))

    plt.figure(figsize=(16,16))
    plt.axis('off')
    plt.title('{}'.format(name))
    for i in range(min(len(corners_1),len(corners_2))):
        # for random line color generation
        r = random.random()
        b = random.random()
        g = random.random()
        color = (r, g, b)
        plt.plot([corners_1[i][0],(corners_2[i][0]+img1.shape[1])],[corners_1[i][1],corners_2[i][1]],color=color,linewidth=0.5)
    plt.imshow(overall_image)

# Function to do 3-D projective reconstruction from rectified images
# Inspired/Derived from previous 2020 HW submissions
def projective_reconstruction(x1,x2,y1,y2,P,P_dash):

    # P,P_dash: left and right camera projection matrices 
    # x1,y1: point correspondences in left image
    # x2,y2: point correspondences in right image
    n = 4
    three_D_vec_array = []
    for i in range(len(x1)):
        D = np.zeros((n,n))
        D[0] = x1[i]*P[2] 
        D[0] = D[0] - P[0]
        D[1] = y1[i]*P[2] 
        D[1] = D[1] - P[1]
        D[2] = x2[i]*P_dash[2]
        D[2] = D[2] - P_dash[0]
        D[3] = y2[i]*P_dash[2] 
        D[3] = D[3] - P_dash[1]

        # to compute world coordinate using one pair of point correspondences
        U,S,V = np.linalg.svd(D)
        X = np.transpose(V[3])
        X = X/X[3]
        three_D_vec_array.append(X)
    three_D_vec_array = np.array(three_D_vec_array)

    # return all world points
    return three_D_vec_array
        
    

In [ ]:
###############################################################################
########################## Main functions of Task 1  ##########################
###############################################################################

# Displaying own stereo images
img1 = cv2.imread ('Task1Images/task_2_1_img7.png')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.imread ('Task1Images/task_2_1_img8.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

# Manually picked left image point correspondences for F and homography estimation
x1_orig = [491,480,820,197,466,200,790,64]
y1_orig = [394,701,484,104,895,776,624,506]

# Show corresponding points on respective stereo images
draw_bounding_box(img1,x1_orig,y1_orig)

# Manually picked right image point correspondences for F and homography estimation
x2_orig = [471,440,794,165,431,169,761,31]
y2_orig = [400,710,485,118,904,790,622,524]

# Show corresponding points on respective stereo images
draw_bounding_box(img2,x2_orig,y2_orig)

# Normalize corresponding points by mean
# Inspired/Derived from previous 2020 HW submissions
x_mean_img_1 = sum(x1_orig)/8
y_mean_img_1 = sum(y1_orig)/8
x_mean_img_2 = sum(x2_orig)/8
y_mean_img_2 = sum(y2_orig)/8
# computer errors w.r.t mean
error1 = np.array(x1_orig) - x_mean_img_1
error2 = np.array(x2_orig) - x_mean_img_2
error3 = np.array(y1_orig) - y_mean_img_1
error4 = np.array(y2_orig) - y_mean_img_2
error_img_1 = np.square(error1) + np.square(error3)
error_img_1 = np.sqrt(error_img_1)
error_img_2 = np.square(error2) + np.square(error4)
error_img_2 = np.sqrt(error_img_2)
c_img_1 = math.sqrt(2)/np.mean(error_img_1)
c_img_2 = math.sqrt(2)/np.mean(error_img_2)
r_img_1 = c_img_1*x_mean_img_1
r_img_2 = c_img_2*x_mean_img_2

# Translational matrices for normalization 
T1 = np.array([[c_img_1,0,-r_img_1], 
               [0,c_img_1,-r_img_1],
               [0,0,1]])
T2 = np.array([[c_img_2,0,-r_img_2], 
               [0,c_img_2,-r_img_2], 
               [0,0,1]])

# Transform all corresponding points using translational matrices for both images
trans_pts_img_1_x = []
trans_pts_img_1_y = []
trans_pts_img_2_x = []
trans_pts_img_2_y = []
for i in range(len(x1_orig)):
     trans_pts_img_1 = np.dot(T1,np.transpose(np.array(([x1_orig[i],y1_orig[i],1]))))
     trans_pts_img_2 = np.dot(T2,np.transpose(np.array(([x2_orig[i],y2_orig[i],1]))))
     trans_pts_img_1 = trans_pts_img_1/trans_pts_img_1[2]
     trans_pts_img_2 = trans_pts_img_2/trans_pts_img_2[2]
     trans_pts_img_1_x.append(trans_pts_img_1[0])
     trans_pts_img_1_y.append(trans_pts_img_1[1])
     trans_pts_img_2_x.append(trans_pts_img_2[0])
     trans_pts_img_2_y.append(trans_pts_img_2[1])

# These are the normalized transformed corresponding points
x1 = trans_pts_img_1_x
y1 = trans_pts_img_1_y
x2 = trans_pts_img_2_x
y2 = trans_pts_img_2_y
p1 = (x1[0],y1[0]); p2 = (x1[1],y1[1]); p3 = (x1[2],y1[2]); p4 = (x1[3],y1[3]); p5 = (x1[4],y1[4])
p6 = (x1[5],y1[5]); p7 = (x1[6],y1[6]); p8 = (x1[7],y1[7])
q1 = (x2[0],y2[0]); q2 = (x2[1],y2[1]); q3 = (x2[2],y2[2]); q4 = (x2[3],y2[3]); q5 = (x2[4],y2[4])
q6 = (x2[5],y2[5]); q7 = (x2[6],y2[6]); q8 = (x2[7],y2[7])
points1 = [p1,p2,p3,p4,p5,p6,p7,p8]
points2 = [q1,q2,q3,q4,q5,q6,q7,q8]

# Compute fundamental matrix using normalized points
input_F = estimate_fundamental_matrix(points1,points2,8)

# condition F to constraint its rank to 2
F = condition_F(input_F)

# Apply normalization transformations
F = np.dot(np.dot(np.transpose(T2),F),T1)

# Normalize fundamental matrix 
F = F/F[2,2] 

# Estimate left and right epipoles, normalize and compute projection matrices
# in canonical form
e,e_dash = estimate_epipoles(F)
e = e/e[2]
e_dash = e_dash/e_dash[2]
P,P_dash = estimate_projection_canonical(e_dash,F)

# convert 3 X 3 Fundamental matrix to a 9 element vector for NL optimization
f = np.ravel(F)

# Apply non-linear optimization to refine F
F_refined = optimize.least_squares(compute_cost_function,f,args=[x1_orig,x2_orig,y1_orig,y2_orig,P,P_dash],method='lm')

# Extract only array component of output
F_refined = F_refined.x

# Reshape vector to form 3 X 3 matrix
F_refined = F_refined.reshape(3,3)
F_refined = F_refined/F_refined[2,2]
print("F_refined",F_refined)

# Estimate left and right epipoles from refined F
e,e_dash = estimate_epipoles(F_refined)
e = e/e[2]
e_dash = e_dash/e_dash[2]

# Estimate left and right projection matrices from refined F
P,P_dash = estimate_projection_canonical(e_dash,F_refined)

# Compute both homographies for rectification
H,H_dash = computing_homographies(img1.shape[0],img1.shape[1],P,P_dash,F,e,e_dash,x1,x2,y1,y2)
print("H",H)
print("H_dash",H_dash)

# Apply homographies to both original images to rectify them
plt.figure()
rect_img_1 = cv2.warpPerspective(img1,H,(img1.shape[0],img1.shape[1]))
plt.imshow(rect_img_1)
rect_img_2 = cv2.warpPerspective(img2,H_dash,(img2.shape[0],img2.shape[1]))
plt.figure()
plt.imshow(rect_img_2)
cv2.imwrite("task_2_recitified_img7_new.png",rect_img_1)
cv2.imwrite("task_2_recitified_img8_new.png",rect_img_2)


In [ ]:
# Plot rectified images 
plt.figure(figsize=(4,4))
plt.imshow(rect_img_1)
plt.axis('off')
plt.figure(figsize=(4,4))
plt.imshow(rect_img_2)
plt.axis('off')

# Detect interest points using Canny Edge Detector
edges_image_1 = canny_edge_detector(rect_img_1)
edges_image_2 = canny_edge_detector(rect_img_2)

# Find point correspondences, select best ones using SSD and plot correspondences
corrs1,corrs2 = find_correspondences(edges_image_1,edges_image_2)

# Plot only 100 point correspondences
index = np.random.choice(len(corrs1),100,replace=False)  
corrs1 = np.array(corrs1)
corrs1 = corrs1[index] 
corrs2 = np.array(corrs2)
corrs2 = corrs2[index]

# Plot point correspondences
draw_matches_harris(corrs1,corrs2,rect_img_1,rect_img_2,'Corresponding points after rectification')

In [ ]:
# manually selected points on rectified images for 3D projective reconstruction
x1_orig = [96,463,829,558,73,400,770,430]
y1_orig = [404,697,567,407,580,885,688,482]
x2_orig = [58,418,809,531,42,363,754,397]
y2_orig = [406,691,565,407,580,884,690,484]

# plot these selected points on rectified images
draw_bounding_box(rect_img_1,x1_orig,y1_orig)
draw_bounding_box(rect_img_2,x2_orig,y2_orig)

# Normalize corresponding points by mean
# Inspired/Derived from previous 2020 HW submissions
x_mean_img_1 = sum(x1_orig)/8
y_mean_img_1 = sum(y1_orig)/8
x_mean_img_2 = sum(x2_orig)/8
y_mean_img_2 = sum(y2_orig)/8
# computer errors w.r.t mean
error1 = np.array(x1_orig) - x_mean_img_1
error2 = np.array(x2_orig) - x_mean_img_2
error3 = np.array(y1_orig) - y_mean_img_1
error4 = np.array(y2_orig) - y_mean_img_2
error_img_1 = np.square(error1) + np.square(error3)
error_img_1 = np.sqrt(error_img_1)
error_img_2 = np.square(error2) + np.square(error4)
error_img_2 = np.sqrt(error_img_2)
c_img_1 = math.sqrt(2)/np.mean(error_img_1)
c_img_2 = math.sqrt(2)/np.mean(error_img_2)
r_img_1 = c_img_1*x_mean_img_1
r_img_2 = c_img_2*x_mean_img_2

# Translational matrices for normalization 
T1 = np.array([[c_img_1,0,-r_img_1], 
               [0,c_img_1,-r_img_1],
               [0,0,1]])
T2 = np.array([[c_img_2,0,-r_img_2], 
               [0,c_img_2,-r_img_2], 
               [0,0,1]])

# Transform all corresponding points using translational matrices for both images
trans_pts_img_1_x = []
trans_pts_img_1_y = []
trans_pts_img_2_x = []
trans_pts_img_2_y = []
for i in range(len(x1_orig)):
     trans_pts_img_1 = np.dot(T1,np.transpose(np.array(([x1_orig[i],y1_orig[i],1]))))
     trans_pts_img_2 = np.dot(T2,np.transpose(np.array(([x2_orig[i],y2_orig[i],1]))))
     trans_pts_img_1 = trans_pts_img_1/trans_pts_img_1[2]
     trans_pts_img_2 = trans_pts_img_2/trans_pts_img_2[2]
     trans_pts_img_1_x.append(trans_pts_img_1[0])
     trans_pts_img_1_y.append(trans_pts_img_1[1])
     trans_pts_img_2_x.append(trans_pts_img_2[0])
     trans_pts_img_2_y.append(trans_pts_img_2[1])

# These are the normalized transformed corresponding points
x1 = trans_pts_img_1_x
y1 = trans_pts_img_1_y
x2 = trans_pts_img_2_x
y2 = trans_pts_img_2_y
p1 = (x1[0],y1[0]); p2 = (x1[1],y1[1]); p3 = (x1[2],y1[2]); p4 = (x1[3],y1[3]); p5 = (x1[4],y1[4])
p6 = (x1[5],y1[5]); p7 = (x1[6],y1[6]); p8 = (x1[7],y1[7])
q1 = (x2[0],y2[0]); q2 = (x2[1],y2[1]); q3 = (x2[2],y2[2]); q4 = (x2[3],y2[3]); q5 = (x2[4],y2[4])
q6 = (x2[5],y2[5]); q7 = (x2[6],y2[6]); q8 = (x2[7],y2[7])
points1 = [p1,p2,p3,p4,p5,p6,p7,p8]
points2 = [q1,q2,q3,q4,q5,q6,q7,q8]

# Compute fundamental matrix using normalized points
input_F = estimate_fundamental_matrix(points1,points2,8)

# condition F to constraint its rank to 2
F = condition_F(input_F)

# Apply normalization transformations
F = np.dot(np.dot(np.transpose(T2),F),T1)

# Normalize fundamental matrix 
F = F/F[2,2] 

# Estimate left and right epipoles, normalize and compute projection matrices
# in canonical form
e,e_dash = estimate_epipoles(F)
e = e/e[2]
e_dash = e_dash/e_dash[2]
P,P_dash = estimate_projection_canonical(e_dash,F)

# convert 3 X 3 Fundamental matrix to a 9 element vector for NL optimization
f = np.ravel(F)

# Apply non-linear optimization to refine F
F_refined = optimize.least_squares(compute_cost_function,f,args=[x1_orig,x2_orig,y1_orig,y2_orig,P,P_dash],method='lm')

# Extract only array component of output
F_refined = F_refined.x

# Reshape vector to form 3 X 3 matrix
F_refined = F_refined.reshape(3,3)
F_refined = F_refined/F_refined[2,2]
print("F_refined",F_refined)

# Estimate left and right epipoles from refined F
e,e_dash = estimate_epipoles(F_refined)
e = e/e[2]
e_dash = e_dash/e_dash[2]

# Estimate left and right projection matrices from refined F
P,P_dash = estimate_projection_canonical(e_dash,F_refined)

# Compute all 3-D world points
t = projective_reconstruction(x1_orig,y1_orig,x2_orig,y2_orig,P,P_dash )


In [ ]:
# point list is used to draw lines between points
point_list = [(0,1),(1,2),(2,3),(0,3),(0,4),(1,5),(2,6),(3,7),(0,4),(1,5),(2,6),(6,7),(4,7),(4,5),(5,6)]

# point correspondences used for 3D reconstruction
x1_orig = [491,480,820,392,466,200,790,64]
y1_orig = [394,701,484,501,895,776,624,506]
x2_orig = [471,440,794,360,431,169,761,31]
y2_orig = [400,710,485,510,904,790,622,524]

# code to plot show 3-D reconstruction and point correspondences of rectified images
fig = plt.figure(figsize=(35,35))
ax = fig.add_subplot(2, 2, 1, projection='3d')
x = t[:,0]
y = t[:,1]
z = t[:,2]
ax.scatter(x,y,z,s=70)
ax.text(t[0][0],t[0][1],t[0][2], "1", color='red',size = 20)
ax.text(t[1][0],t[1][1],t[1][2], "2", color='red',size = 20)
ax.text(t[2][0],t[2][1],t[2][2], "3", color='red',size = 20)
ax.text(t[3][0],t[3][1],t[3][2], "4", color='red',size = 20)
ax.text(t[4][0],t[4][1],t[4][2], "5", color='red',size = 20)
ax.text(t[5][0],t[5][1],t[5][2], "6", color='red',size = 20)
ax.text(t[6][0],t[6][1],t[6][2], "7", color='red',size = 20)
ax.text(t[7][0],t[7][1],t[7][2], "8", color='red',size = 20)
ax.view_init(250,150)
# code to draw lines between recontructed points
for i in point_list:
    x = i[0]
    y = i[1]
    ax.plot([t[x][0],t[y][0]],[t[x][1],t[y][1]],[t[x][2],t[y][2]])
ax = fig.add_subplot(2, 2, 2)
ax.axis('off')
ax.imshow(rect_img_1)
ax = fig.add_subplot(2, 2, 3)
ax.axis('off')
ax.imshow(rect_img_2)

# code to plot different views of 3-D reconstruction
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1,projection='3d')
x = t[:,0]
y = t[:,1]
z = t[:,2]
ax.scatter(x,y,z,s=70)
ax.text(t[0][0],t[0][1],t[0][2], "1", color='red',size = 20)
ax.text(t[1][0],t[1][1],t[1][2], "2", color='red',size = 20)
ax.text(t[2][0],t[2][1],t[2][2], "3", color='red',size = 20)
ax.text(t[3][0],t[3][1],t[3][2], "4", color='red',size = 20)
ax.text(t[4][0],t[4][1],t[4][2], "5", color='red',size = 20)
ax.text(t[5][0],t[5][1],t[5][2], "6", color='red',size = 20)
ax.text(t[6][0],t[6][1],t[6][2], "7", color='red',size = 20)
ax.text(t[7][0],t[7][1],t[7][2], "8", color='red',size = 20)
ax.view_init(250,50)
# code to draw lines between recontructed points
for i in point_list:
    x = i[0]
    y = i[1]
    ax.plot([t[x][0],t[y][0]],[t[x][1],t[y][1]],[t[x][2],t[y][2]])

In [ ]:
# ###############################################################################
# ########################## Task 3  ############################################
# ###############################################################################

def census_transform (left_img,right_img,M,d_max) :
    disparity_map = np.zeros((left_img.shape[0],left_img.shape[1]))
    for i in range ( d_max + int(M/2),left_img.shape[0]-int(M/2)-d_max) :
        for j in range (left_img.shape[1]-int(M/2)-1-d_max,d_max + int(M/2)-1,-1) :
            a = []
            left_image_window = left_img[i-int(M/2):i+1+int(M/2),j-int(M/2):j+1+int(M/2)]
            L_array = left_image_window > left_img[i,j]
            left_bit_vec = np.ravel (L_array.astype(int))
            for d in range (0,d_max+1) :
                right_image_window = right_img [i-int(M/2):i+1+int(M/2),j-d-int(M/2):j-d+1+int(M/2)]
                R_array = right_image_window > right_img [i,j-d]
                right_bit_vec = np.ravel(R_array.astype(int))
                result_bit_array = np.logical_xor(left_bit_vec,right_bit_vec)
                unique, counts = np.unique(result_bit_array,return_counts=True)
                counter = dict(zip(unique, counts))
                if 1 in counter :
                    a.append(counter[1])
                else : 
                    a.append(0)
            disparity_map[i,j] = np . argmin (a)
    disparity_map = np.uint8(disparity_map)
    disparity_map_scaled = disparity_map / np.max(disparity_map)
    disparity_map_scaled = np.uint8(disparity_map_scaled*255)
    return disparity_map,disparity_map_scaled

def compute_accuracy_dmap(gt_d_map,computed_d_map):

    comp_dmap = np.int16(computed_d_map)
    gt_dmap = np.int16(gt_d_map)
    error = np.uint8(abs(comp_dmap-gt_d_map))
    valid_mask = 255 * (error <= 2)
    valid_mask = np.uint8(valid_mask)
    pos = np.bitwise_and(computed_d_map,valid_mask)
    non_zero = np.count_nonzero(computed_d_map)
    acc = np.count_nonzero(pos)/non_zero
    for i in range(pos.shape[0]):
        for j in range(pos.shape[1]):
            if pos[i,j] !=0:
                pos[i,j] = 255
    return acc,pos

left_img = cv2.imread ('Task3Images/im2.png')
left_img = cv2.cvtColor(left_img,cv2.COLOR_BGR2GRAY)
right_img = cv2.imread ('Task3Images/im6.png')
right_img = cv2.cvtColor(right_img,cv2.COLOR_BGR2GRAY)
plt.figure(figsize=(6,6))
plt.imshow(left_img,cmap="gray")
plt.axis('off')
plt.title("Left stereo image")
plt.figure(figsize=(6,6))
plt.imshow(right_img,cmap="gray")
plt.axis('off')
plt.title("Right stereo image")

left_gt_img = cv2.imread ('Task3Images/disp2.png')
left_gt_img = cv2.cvtColor(left_gt_img,cv2.COLOR_BGR2GRAY)
plt.figure(figsize=(6,6))
plt.imshow(left_gt_img,cmap="gray")
plt.axis('off')
plt.title("Ground truth Disparity Map in grayscale")
left_gt_img = np.float32(left_gt_img)
left_gt_img = left_gt_img/4
left_gt_img = np.uint8(left_gt_img)

d_out_win_30_d_52,d_out_win_30_d_52_scaled = census_transform(left_img,right_img,30,52)
plt.figure(figsize=(6,6))
plt.imshow(d_out_win_30_d_52_scaled,cmap="gray")
plt.title("Computed Disparity Map in grayscale for Window 30 d_max = 52")
plt.axis("off")

acc,valid_mask = compute_accuracy_dmap(left_gt_img,d_out_win_30_d_52)
print("Accuracy",acc)
plt.figure()
plt.imshow(valid_mask,cmap="gray")
plt.title("Binary error mask for Window 30 d_max = 52")
plt.axis("off")

d_out_win_11_d_52,d_out_win_11_d_52_scaled = census_transform(left_img,right_img,11,52)
plt.figure(figsize=(6,6))
plt.imshow(d_out_win_11_d_52_scaled,cmap="gray")
plt.title("Computed Disparity Map in grayscale for Window 11 d_max = 52")
plt.axis("off")

acc,valid_mask = compute_accuracy_dmap(left_gt_img,d_out_win_11_d_52)
print("Accuracy",acc)
plt.figure()
plt.imshow(valid_mask,cmap="gray")
plt.title("Binary error mask for Window 11 d_max = 52")
plt.axis("off")

d_out_win_49_d_52,d_out_win_49_d_52_scaled = census_transform(left_img,right_img,49,52)
plt.figure(figsize=(6,6))
plt.imshow(d_out_win_49_d_52_scaled,cmap="gray")
plt.title("Computed Disparity Map in grayscale for Window 49 d_max = 52")
plt.axis("off")

acc,valid_mask = compute_accuracy_dmap(left_gt_img,d_out_win_49_d_52)
print("Accuracy",acc)
plt.figure()
plt.imshow(valid_mask,cmap="gray")
plt.title("Binary error mask for Window 49 d_max = 52")
plt.axis("off")